In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [2]:
import sys
sys.path.append("/workspaces/remember-me/back/src")


In [3]:
from remember.batch.generate_event_memories import generate_event_memories, save_event_memories, load_event_memories
from remember.batch.generate_day_memory import generate_day_memory, load_day_memory, save_day_memory

In [13]:
event_memories = generate_event_memories(event = "Les vacances à Rome", 
                  name = "Monique",
                  age = 75,
                  occupation = "Ancienne Assistante sociale",
                  nb_memories = 15) 

event_memories.model_dump()

KeyboardInterrupt: 

In [ ]:
save_event_memories(event_memories,"event_memories.json")

In [ ]:
event_memories = load_event_memories("event_memories.json")

In [ ]:
day_memory = generate_day_memory(name = "Monique",
                  age = 75,
                  occupation = "Ancienne Assistante sociale",
                  memories = event_memories)

In [ ]:
save_day_memory(day_memory,"day_memory.json")

In [ ]:
day_memory = load_day_memory("day_memory.json")

In [ ]:
day_memory

In [4]:
from langchain.chat_models import init_chat_model
from elevenlabs.client import ElevenLabs
from elevenlabs import play, save
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from IPython.display import display, Markdown
from typing import List
from enum import Enum
import os

from pydantic import BaseModel, Field
import json
from typing import List

In [5]:
#model = init_chat_model("ministral-8b-latest", model_provider="mistralai", temperature=0)
#model = init_chat_model("mistral-large-latest", model_provider="mistralai", temperature=0.2)
model = init_chat_model("gpt-4o-mini", model_provider="openai", temperature=0.3, max_retries=3)

In [6]:
class Period(str, Enum):
    """Enum for periods of life."""
    CHILDHOOD = "childhood"
    ADOLESCENCE = "adolescence"
    YOUNG_ADULTHOOD = "young_adulthood"
    ADULTHOOD = "adulthood"
    OLD_AGE = "old_age"

class Person(BaseModel):
    id: int = Field(description="The unique identifier for the person")
    name: str = Field(description="The name of the person")
    relationship: str = Field(description="The relationship of the person to the user")
    age: int = Field(description="The age of the person")
    occupation: str = Field(description="The occupation of the person")
    hobbies: List[str] = Field(default_factory=list, description="The hobbies of the person")
    description: str = Field(description="A description of the person")

    def __str__(self):
        return json.dumps(self.dict(), indent=2)

class Memory(BaseModel):
    people: List[int] = Field(min_items=1, decription="A list of IDs of people involved in the memory")
    place: List[str] = Field(min_items=1, default_factory=list, description="A list of places involved in the memory")
    period: List[str] = Field(min_items=1, default_factory=list)
    emotion: List[str] = Field(min_items=1, default_factory=list)
    description: str = Field(description="A description of the memory")

    def __str__(self):
        return json.dumps(self.dict(), indent=2)
    
class LifeSummary(BaseModel):
    name: str = Field(description="The name of the user")
    age: int = Field(description="The age of the user")
    occupation: str = Field(description="The occupation of the user")
    acquaintances: List[Person] = Field(min_items=5, default_factory=list, description="A list of acquaintances of the user (friend, colleague,  family)")
    memories: List[Memory] = Field(default_factory=list)

    def __str__(self):
        return json.dumps(self.dict(), indent=2)

# Génération avec le nouveau modele de donnes

In [6]:
from remember.batch.db import DB

In [7]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", """Tu es un générateur données de synthèse, qui intègrent des souvenirs narratifs, capable de simuler différents styles d’écriture, 
                      tons émotionnels et points de vue. Tu adoptes en particulier le rôle de plusieurs proches d'une personne agée pour raconter leurs souvenirs."""),
        ("human", """
                      **Contexte de la personne agée** :
                        - Nom : {name}
                        - Âge : {age}
                        - Ancienne profession : {occupation}
                        - Elle a une mémoire défficiente, et ses proches souhaitent lui faire revivre des moments heureux à travers ces souvenirs.
                     
                     Les personnes à générer sont :
                    - fils
                    - belle fils
                    - petit fils
                    - petite fille
                    - frère
                    - mari

                    Les évènements à générer sont :
                    - l'anniversaire du fils
                    - le mariage du petit fils

                     Rempli une base de données imaginaire selon le schema demandé.
                     """)
    ]
)
chain = prompt_template | model.with_structured_output(DB)
life_summary = chain.invoke({"name": "Monique", "age": "75", "occupation": "Ancienne Assistante sociale"})

In [20]:
db_json = life_summary.model_dump()

In [60]:
person_dict = {}
for person in life_summary.persons:
    person_json = person.model_dump()
    del person_json["annecdotes"]
    del person_json["profile_picture_id"]
    person_dict[person.id] = person_json
person_dict

{1: {'id': 1,
  'name': 'Pierre',
  'surname': 'Dupont',
  'birthdate': '1980-05-15',
  'description': 'Fils de Monique, il a toujours été très proche de sa mère.',
  'relationship_type': <RelationshipType.FAMILY: 'family'>,
  'relationship': 'fils'},
 2: {'id': 2,
  'name': 'Sophie',
  'surname': 'Martin',
  'birthdate': '1985-03-22',
  'description': 'Belle-fille de Monique, elle a toujours été accueillie avec chaleur par sa belle-mère.',
  'relationship_type': <RelationshipType.FAMILY: 'family'>,
  'relationship': 'belle-fille'},
 3: {'id': 3,
  'name': 'Lucas',
  'surname': 'Dupont',
  'birthdate': '2010-07-30',
  'description': 'Petit-fils de Monique, il adore passer du temps avec sa grand-mère.',
  'relationship_type': <RelationshipType.FAMILY: 'family'>,
  'relationship': 'petit-fils'},
 4: {'id': 4,
  'name': 'Emma',
  'surname': 'Dupont',
  'birthdate': '2012-11-12',
  'description': 'Petite-fille de Monique, elle adore passer du temps avec sa grand-mère.',
  'relationship_typ

In [56]:
event = life_summary.events[0]

In [57]:
event = event.model_dump()

In [58]:
event["people"] = [person_dict[person_id] for person_id in event["person_id"]]
del event[id]
del event["person_id"]

KeyError: <built-in function id>

In [59]:
event

{'id': 1,
 'name': 'Anniversaire de Pierre',
 'date': '2023-05-15',
 'description': 'Célébration des 43 ans de Pierre avec toute la famille.',
 'annecdotes': ["Monique a préparé son gâteau au chocolat préféré pour l'occasion.",
  "Tout le monde a chanté 'Joyeux Anniversaire' avec enthousiasme.",
  'Pierre a reçu un cadeau surprise de sa mère, un album photo de son enfance.',
  'Nous avons joué à des jeux de société, Monique était la championne.',
  "Elle a partagé des histoires drôles de l'enfance de Pierre.",
  'Les rires ont résonné dans la maison, créant une ambiance chaleureuse.',
  'Monique a dansé avec Pierre sur une chanson de leur époque.',
  'Elle a pris des photos de chaque moment, capturant la joie de la journée.',
  "Nous avons fait un toast en l'honneur de Pierre, c'était émouvant.",
  'Monique a dit à Pierre combien elle était fière de lui.'],
 'person_id': [1, 2, 3, 4, 5, 6],
 'people': [{'id': 1,
   'name': 'Pierre',
   'surname': 'Dupont',
   'birthdate': '1980-05-15',

In [28]:
life_summary.events[0]

Event(id=1, name='Anniversaire de Pierre', date='2023-05-15', description='Célébration des 43 ans de Pierre avec toute la famille.', annecdotes=["Monique a préparé son gâteau au chocolat préféré pour l'occasion.", "Tout le monde a chanté 'Joyeux Anniversaire' avec enthousiasme.", 'Pierre a reçu un cadeau surprise de sa mère, un album photo de son enfance.', 'Nous avons joué à des jeux de société, Monique était la championne.', "Elle a partagé des histoires drôles de l'enfance de Pierre.", 'Les rires ont résonné dans la maison, créant une ambiance chaleureuse.', 'Monique a dansé avec Pierre sur une chanson de leur époque.', 'Elle a pris des photos de chaque moment, capturant la joie de la journée.', "Nous avons fait un toast en l'honneur de Pierre, c'était émouvant.", 'Monique a dit à Pierre combien elle était fière de lui.'], person_id=[1, 2, 3, 4, 5, 6])

In [21]:
db_json.persons[0]

AttributeError: 'dict' object has no attribute 'persons'

In [15]:
with open("db.json", "w") as f:
    f.write(db_json)

# Génération des souvenirs pour la vie

In [ ]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage("Tu est un chatbot qui génère des données de synthèse sur les souvenirs sur toute la vie d'une personne agée."),
        ("user", """Invente cinq souvenirs. pour une personne dont voici la description :
            Nom : {name}
            Age : {age}
            Profession : {occupation} 
         Rédige les souvenirs à la première personne,

         en utilisant le vocabulaire et les émotions d'une personne âgée.
         Chaque souvenir doit être spécifique et détaillé, en incluant la référence (id) des personnes impliquées, les lieux, la période de la vie et les émotions ressenties."""),
    ]
)
chain = prompt_template | model.with_structured_output(LifeSummary)

In [ ]:
life_summary = chain.invoke({"name": "Monique", "age": "75", "occupation": "Ancienne Assistante sociale"})

In [ ]:
life_summary.model_dump()

# Génération de souvenirs pour un évènement particulier

In [ ]:
class Person(BaseModel):
    name: str = Field(description="The name of the person")
    relationship: str = Field(description="The relationship of the person to the user")
    age: int = Field(description="The age of the person")
    occupation: str = Field(description="The occupation of the person")
    hobbies: List[str] = Field(default_factory=list, description="The hobbies of the person")
    description: str = Field(description="A description of the person")

    def __str__(self):
        return json.dumps(self.dict(), indent=2)

class Memory(BaseModel):
    person: Person = Field(decription="The writer of the memory")
    place: List[str] = Field(default_factory=list, description="A list of places involved in the memory")
    emotion: List[str] = Field(default_factory=list)
    description: str = Field(description="A description of the memory")

    def __str__(self):
        return json.dumps(self.dict(), indent=2)
    
class EnventMemories(BaseModel):
    """Memories collection about a single event with different perspectives."""
    event: str = Field(description="The specific event being remembered")
    memories: List[Memory] = Field(default_factory=list)

    def __str__(self):
        return json.dumps(self.dict(), indent=2)

In [ ]:
prompt_template = ChatPromptTemplate(
    [
        ("system", """Tu es un générateur de souvenirs narratifs, capable de simuler différents styles d’écriture, 
                      tons émotionnels et points de vue. Tu adoptes le rôle de plusieurs proches d'une personne agée pour raconter leurs souvenirs de leur point de vue pour un même évènement."""),
        ("human", """Invente {nb_memories} souvenirs riches, spécifiques et émotionnels racontés par différents proches de la personne agée et liés à un même évènement ‘‘‘{event}‘‘‘. 
                      Chaque souvenir doit être rédigé à la première personne, dans un style adapté à la personnalité du narrateur.

                      **Contexte de la personne agée** :
                        - Nom : {name}
                        - Âge : {age}
                        - Ancienne profession : {occupation}
                        - Elle a une mémoire défficiente, et ses proches souhaitent lui faire revivre des moments heureux à travers ces souvenirs.
                     
                     Rédige les souvenirs à la première personne au sujet de l'évènement : ‘‘‘{event}‘‘‘. 
                     Chaque souvenir est rédigé par un proche distinct (par exemple amis, enfants, petits-enfants, voisins, collègues retraités, etc.).
                     Chaque souvenir doit être spécifique et détaillées et en intégrant les émotions ressenties. Utilise des anecdotes.
                     """
        )
    ]
)
chain = prompt_template | model.with_structured_output(EnventMemories)

In [ ]:
event_memories = chain.invoke({"name": "Monique",
"age": "75",
"occupation": "Ancienne Assistante sociale",
"event": "Mariage de Thibault, le petit fils de Monique",
"nb_memories": 15})

In [ ]:
event_memories.model_dump()

In [ ]:
with open("event_memories.json", "w", encoding="utf-8") as file:
    file.write(event_memories.model_dump_json(indent=2))

# Synthèse de l'histoire

In [ ]:
with open("event_memories.json", "r", encoding="utf-8") as file:
    event_memories = json.load(file)

In [ ]:
day_memory

In [ ]:
display(Markdown(day_memory.summary))

In [ ]:

# Initialize ElevenLabs client with your API key
api_key = os.getenv("ELEVENLABS_API_KEY")
client = ElevenLabs(api_key=api_key)
text = day_memory.summary
voice_id = "sSSnx1yBXjhBo6gzmfyI"

# Generate audio from text using the specified voice ID
audio = client.generate(
    text=text,
    voice=voice_id,
    model="eleven_multilingual_v2",
    output_format="mp3_44100_128",
    voice_settings={
        "stability": 0.70,
        "similarity_boost": 0.75
    }
)

output_path = "output.mp3"

# Save the generated audio to a file
save(audio, output_path)
print(f"Audio saved to {output_path}")